# European Union Energy Map

Dashboard with interactive map visualizing Eurostat data regarding renewable energy developments in the European Union.

Data is pulled from [Eurostat](https://ec.europa.eu/eurostat/databrowser/explore/all/envir?lang=en&subtheme=nrg&display=list&sort=category).

**GitHub**: [https://github.com/kuranez/EU-Energy-Map](https://github.com/kuranez/EU-Energy-Map)

# Table of Contents
1. [Data Sources](#data-sources)
2. [Imports](#imports)
3. [Settings](#settings)
4. [Load & Format Data](#load-&-format-data)
5. [List Countries & Energy Types](#list-countries-&-energy-types)
6. [Filter Data to Display](#filter-data-to-display)
7. [Dashboard Components](#dashboard-components)
8. [Dashboard Content](#dashboard-content)
9. [Create Dashboard](#create-dashboard)


## Data Sources

### List of Datasets

**nrg_ind_ren_linear.csv**

- **Share of energy from renewable sources**
- **Source:** https://ec.europa.eu/eurostat/databrowser/view/nrg_ind_ren/default/table?lang=en

## Imports

In [2]:
import os
import json
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from plotly.colors import sample_colorscale
import plotly.express as px
import plotly.io as pio
import panel as pn

## Settings

### Panel Extension

In [3]:
# Set up Panel extension
pn.extension('tabulator', 'plotly', design='material', sizing_mode='stretch_width')

### Plotly Renderer

In [4]:
# Set Plotly renderer to JupyterLab
# pio.renderers.default = "jupyterlab"

### MapBox

In [5]:
# Set MapBox-Access-Token
mapbox_token = 'pk.eyJ1Ijoia3VyYW5leiIsImEiOiJjbTJmMjI0d2kwNDVxMnFzYXNldnc1N2VsIn0.t11TYpF2QBdid-hQfW8mig'

## Load & Format Data

### Load EuroStat-Data

In [6]:
# Load Data
data = pd.read_csv('./data/nrg_ind_ren_linear.csv')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     2772 non-null   object 
 1   LAST UPDATE  2772 non-null   object 
 2   freq         2772 non-null   object 
 3   nrg_bal      2772 non-null   object 
 4   unit         2772 non-null   object 
 5   geo          2772 non-null   object 
 6   TIME_PERIOD  2772 non-null   int64  
 7   OBS_VALUE    2772 non-null   float64
 8   OBS_FLAG     0 non-null      float64
dtypes: float64(2), int64(1), object(6)
memory usage: 195.0+ KB


### Load GeoJSON

In [8]:
# Load GeoJSON for European countries
with open('europe.geojson') as f:
    europe_map = json.load(f)

### Merge Data

In [9]:
# Merge energy data with map data
merged_data = gpd.read_file('europe.geojson').merge(data, left_on='ISO2', right_on='geo')

In [10]:
merged_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2544 entries, 0 to 2543
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   FID          2544 non-null   float64 
 1   FIPS         2544 non-null   object  
 2   ISO2         2544 non-null   object  
 3   ISO3         2544 non-null   object  
 4   UN           2544 non-null   int32   
 5   NAME         2544 non-null   object  
 6   AREA         2544 non-null   int32   
 7   POP2005      2544 non-null   int32   
 8   REGION       2544 non-null   int32   
 9   SUBREGION    2544 non-null   int32   
 10  LON          2544 non-null   float64 
 11  LAT          2544 non-null   float64 
 12  geometry     2544 non-null   geometry
 13  DATAFLOW     2544 non-null   object  
 14  LAST UPDATE  2544 non-null   object  
 15  freq         2544 non-null   object  
 16  nrg_bal      2544 non-null   object  
 17  unit         2544 non-null   object  
 18  geo          2544 no

### Clean Data

In [11]:
# Clean and rename columns for readability
merged_data = merged_data.rename(columns={
    'nrg_bal': 'Energy Type',
    'TIME_PERIOD': 'Year',
    'OBS_VALUE': 'Renewable Percentage',
    'geo': 'Country Code',
    'NAME': 'Country',
})

In [12]:
# Rename entries in 'Energy Type' column for clarity
merged_data['Energy Type'] = merged_data['Energy Type'].replace({
    'REN': 'Renewable Energy Total',
    'REN_ELC': 'Renewable Electricity',
    'REN_HEAT_CL': 'Renewable Heating and Cooling',
    'REN_TRA': 'Renewable Energy in Transport'
})

In [13]:
# Drop columns
merged_data = merged_data.drop(columns=['LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'])

### Data Conversion

In [14]:
# Convert to numeric
merged_data[['Year', 'Renewable Percentage', 'AREA', 'POP2005']] = merged_data[['Year', 'Renewable Percentage', 'AREA', 'POP2005']].apply(pd.to_numeric)

# Round the 'Renewable Percentage' to 1 decimal place
merged_data['Renewable Percentage'] = merged_data['Renewable Percentage'].round(1)

### Add Country Flags

In [15]:
@pn.cache
# Function to convert ISO2 country code to flag emoji
def iso2_to_flag(iso2_code):
    # Convert ISO2 code to country flag emoji using the formula
    return chr(0x1F1E6 + ord(iso2_code[0]) - ord('A')) + chr(0x1F1E6 + ord(iso2_code[1]) - ord('A'))

# Add a new column 'Country Flag' with the corresponding country flag emojis
merged_data['Country Flag'] = merged_data['Country Code'].apply(iso2_to_flag)

### Sort Columns

In [16]:
# Reorder columns
final_columns_order = [
    'Country Code', 'Country Flag', 'Country', 'Energy Type', 'Renewable Percentage', 'Year',
    'DATAFLOW', 'FID', 'FIPS', 'ISO2', 'ISO3', 'AREA', 'POP2005', 'REGION', 'SUBREGION', 'geometry'
]

### Final DataFrame

In [17]:
merged_data_clean = merged_data[final_columns_order]

In [18]:
merged_data_clean.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2544 entries, 0 to 2543
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Country Code          2544 non-null   object  
 1   Country Flag          2544 non-null   object  
 2   Country               2544 non-null   object  
 3   Energy Type           2544 non-null   object  
 4   Renewable Percentage  2544 non-null   float64 
 5   Year                  2544 non-null   int64   
 6   DATAFLOW              2544 non-null   object  
 7   FID                   2544 non-null   float64 
 8   FIPS                  2544 non-null   object  
 9   ISO2                  2544 non-null   object  
 10  ISO3                  2544 non-null   object  
 11  AREA                  2544 non-null   int32   
 12  POP2005               2544 non-null   int32   
 13  REGION                2544 non-null   int32   
 14  SUBREGION             2544 non-null   int32   
 

### Explore DataFrame

In [19]:
df_widget = pn.widgets.Tabulator(
    merged_data_clean[['Country Code', 'Country Flag', 'Country', 'Renewable Percentage', 'Year', 'Energy Type']],
    layout='fit_data_stretch',
    pagination="remote",            # Enable pagination, options: local, remote
    page_size=10,                  # Set number of rows per page
    show_index=False,              # Hide Index
    configuration={"movableColumns": False},  # Disable column reordering

    
)

In [20]:
df_widget

BokehModel(combine_events=True, render_bundle={'docs_json': {'497b5fad-cf16-4ffe-9159-5e5ce85e6347': {'version…

## List Countries & Energy Types

### All Countries

In [21]:
# Display list of countries
countries_md = "**Countries:**\n"
for iso2, country in merged_data_clean[['Country Code', 'Country']].drop_duplicates().values:
    countries_md += f"- **{iso2}**: {country}\n"
    
countries_pane = pn.pane.Markdown(countries_md)

countries_pane

BokehModel(combine_events=True, render_bundle={'docs_json': {'909eb64b-8846-4413-be76-23bfff7c9c17': {'version…

### EU-Countries

In [22]:
# EU countries filter
eu_countries = [
    "AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HU", "IE", "IT", "LV", "LT", 
    "LU", "MT", "NL", "PL", "PT", "RO", "SK", "SI", "ES", "SE"
]

eu_data = merged_data_clean[merged_data_clean['Country Code'].isin(eu_countries)]

eu_countries_md = "**EU Countries:**\n"
for iso2, country in eu_data[['Country Code', 'Country']].drop_duplicates().values:
    eu_countries_md += f"- **{iso2}**: {country}\n"

eu_countries_pane = pn.pane.Markdown(eu_countries_md)

eu_countries_pane

BokehModel(combine_events=True, render_bundle={'docs_json': {'9bcd2d09-0a4e-4c4a-ab95-d207f32d8a68': {'version…

### Display Energy Types

In [23]:
# Display Energy Types
energy_types = merged_data_clean['Energy Type'].unique()

energy_types_list = "\n".join([f"- {value}" for value in energy_types])

print(f"Energy Types: {len(energy_types)}\n{energy_types_list}")

Energy Types: 4
- Renewable Energy Total
- Renewable Electricity
- Renewable Heating and Cooling
- Renewable Energy in Transport


## Filter Data to Display

In [24]:
# Renewable Energy Total
df_renewable = merged_data_clean[
                                (merged_data_clean['Energy Type'] == 'Renewable Energy Total') &
                                (merged_data_clean['Country Code'].isin(eu_countries))]

# Year
selected_year = 2022 # for testing/initial display
df_year = df_renewable[df_renewable['Year'] == selected_year]

# Country
selected_country = "Germany" # for testing/iitial display
df_country = df_renewable[df_renewable['Country'] == selected_country]

# EU Total
# - Group by year and calculate the average renewable energy percentage for the EU total
df_eu_total = df_renewable.groupby('Year', as_index=False)['Renewable Percentage'].mean()

## Dashboard Components

### Input-Widgets

#### Year Selection

In [25]:
# Year Selection
year_slider = pn.widgets.IntSlider(name='Select Year', start=2004, end=2022, step=1, value=2022)

In [26]:
# Display & test year slider
year_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'91837678-09d5-4883-9dcd-b2c803ad1212': {'version…

#### Country Selection

In [27]:
# Extract unique country names
unique_countries = df_renewable['Country'].unique().tolist()
unique_flags = df_renewable['Country Flag'].unique().tolist()

# Combine country names and flags
country_flag_items = [f"{flag} {country}" for country, flag in zip(unique_countries, unique_flags)]

# Define the default country
default_country = "Germany"

# Find the corresponding entry in `country_flag_items`
default_value = next(item for item in country_flag_items if default_country in item)

# Country selection widget
country_selection = pn.widgets.Select(name='Select country', options=country_flag_items, value=default_value)

In [28]:
# Display & test country selection
country_selection

BokehModel(combine_events=True, render_bundle={'docs_json': {'ef3c2fb2-01e2-44e1-8ebf-1b02d06c6fd6': {'version…

### Sidebar

In [29]:
# Define Markdown content
sidebar_content = """
#### About
Explore renewable energy developments in the European Union. Data is sourced from [Eurostat](https://ec.europa.eu/eurostat/databrowser/explore/all/envir?lang=en&subtheme=nrg&display=list&sort=category).

#### Project Page on GitHub
[https://github.com/kuranez/EU-Energy-Map](https://github.com/kuranez/EU-Energy-Map) \
"""

sidebar_pane = pn.pane.Markdown(sidebar_content)

In [30]:
sidebar_pane

BokehModel(combine_events=True, render_bundle={'docs_json': {'53208bd9-39ae-4561-b32c-2eeccb0e5e59': {'version…

## Dashboard Content

### Table

In [31]:
# Display data in Tabulator widget
fig_table = pn.widgets.Tabulator(
    df_renewable[['Country', 'Country Code', 'Country Flag', 'Energy Type', 'Renewable Percentage', 'Year']],
    pagination="remote",          # Enables pagination
    page_size=10,                 # Sets the number of rows per page
    show_index=False              # Hides the index column
)

In [32]:
fig_table

BokehModel(combine_events=True, render_bundle={'docs_json': {'60e46c48-55c1-49b3-b1a9-9908e2f3ede7': {'version…

### Map

In [33]:
@pn.cache
# Create Choropleth map function
def create_choropleth_map(df_year):
    fig_map = go.Figure(go.Choroplethmapbox(
        geojson=europe_map,
        locations=df_year['Country Code'],
        z=df_year['Renewable Percentage'],
        colorscale="Viridis",
        zmin=0, zmax=100,
        marker_opacity=0.8, marker_line_width=0.5,
        featureidkey="properties.ISO2",
        hoverinfo='location+z', hovertext=df_year['Renewable Percentage']
    ))
    fig_map.update_layout(
        mapbox_accesstoken=mapbox_token,
        mapbox_style="carto-positron",
        mapbox_zoom=3,
        mapbox_center={"lat": 54, "lon": 15},
        title="Renewable Energy as Percentage by Country in 2022",
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        geo=dict(scope='europe', showlakes=False)
    )
    return fig_map

In [34]:
eu_map = create_choropleth_map(df_year)

/tmp/ipykernel_35158/3207055277.py:4: DeprecationWarning: *choroplethmapbox* is deprecated! Use *choroplethmap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig_map = go.Figure(go.Choroplethmapbox(


In [ ]:
# Display as Panel pane
map_pane = pn.pane.Plotly(eu_map, width=1200)
map_pane

BokehModel(combine_events=True, render_bundle={'docs_json': {'c4bac3bb-22e2-4240-bf13-ee7c64a3b666': {'version…

### Charts

#### Filtered by Year

In [36]:
# Create bar chart by year for all countries
@pn.cache
def create_bar_chart_year(df_year, year):
    # Sort dataframe by renewable percentage in ascending order
    df_year = df_year.sort_values(by='Renewable Percentage')

    # Calculate EU total average renewable percentage for 2022
    eu_total_avg = df_year['Renewable Percentage'].mean()

    # Map EU total average to a color in the Viridis scale
    color_scale = px.colors.sequential.Viridis
    normalized_avg = eu_total_avg / 100  # Normalize average to a 0-1 range
    scaled_color = sample_colorscale(color_scale, normalized_avg)[0]
    
    # Create bar trace (main layer)
    bar_trace = go.Bar(
        x=df_year['Country'],
        y=df_year['Renewable Percentage'],
        marker=dict(
            color=df_year['Renewable Percentage'],
            coloraxis='coloraxis',  # Link to coloraxis
        ),
        name="",  # Fix to hide trace info
        hovertemplate="Country: %{customdata[0]} "
                      "<b>%{customdata[1]}</b><br>"
                      "Renewable Percentage: <b>%{y:.1f}%</b>",
        customdata=df_year[['Country Flag', 'Country']].values,  # Attach custom data
        showlegend=False,
    )
    
    # Add scatter trace for EU Total Average (border layer)
    scatter_trace_border = go.Scatter(
        x=df_year['Country'],  # Use all countries to make the line span across
        y=[eu_total_avg] * len(df_year),  # Duplicate average value for each country
        mode="lines",
        line=dict(dash="solid", color="rgba(255, 255, 255, 0.7)", width=4),
        hoverinfo="skip",  # Suppress hover for border layer
        showlegend=False,
    )
    
    # Add scatter trace for EU Total Average (main layer)
    scatter_trace = go.Scatter(
        x=df_year['Country'],  # Use all countries to make the line span across
        y=[eu_total_avg] * len(df_year),  # Duplicate average value for each country
        mode="lines",
        line=dict(dash="solid", color=scaled_color, width=2),
        hovertemplate="🇪🇺 EU Total Average:<b> %{y:.1f}%</b>",
        name="",  # Fix to suppress showing trace info
        showlegend=False,
    )

    # Create figure
    fig = go.Figure(data=[bar_trace, scatter_trace_border, scatter_trace])

    # Update layout
    fig.update_layout(
        title=f"Renewable Energy Percentage by Country in {year}",
        xaxis=dict(title=None),
        yaxis=dict(title="Renewable Energy (%)"),
        coloraxis=dict(  # Define coloraxis for color bar
            colorscale='Viridis',
            cmin=0,
            cmax=100,
            colorbar=dict(
                orientation="v",
                title=None,
                tickvals=[0, 20, 40, 60, 80, 100],
                ticktext=["0%", "20%", "40%", "60%", "80%", "100%"],
            ),
        ),
        margin={"t": 50, "b": 50, "l": 50, "r": 50},
        height=450,
    )

    return fig

In [37]:
# Create Figure
year = selected_year
fig_by_year = create_bar_chart_year(df_year, year)

In [ ]:
# Display as Panel pane
fig_pane_1 = pn.pane.Plotly(fig_by_year, width=1200)
fig_pane_1

BokehModel(combine_events=True, render_bundle={'docs_json': {'31844b71-647d-4b87-a57d-a76f380ed9f9': {'version…

##### Extras

In [39]:
# # Extras: Annotations

# # Add annotation for EU average
# fig.add_annotation(
#     x=0 + 3.5, # Offset right
#     y=eu_total_avg + 5,  # Offset above the EU average line
#     xref="x",
#     yref="y",
#     text=f"🇪🇺 EU Total Average: <b>{eu_total_avg:.1f} %</b>",
#     showarrow=False,  # No arrow needed
#     align="left",  # Left-align text
#     font=dict(
#         size=12,
#         color="white",
#     ),
#     bgcolor=scaled_color,  # Color-scaled background
#     bordercolor="white",  # Border around the text box
#     borderwidth=1,
#     borderpad=5,
# )

#### Filtered by Country

In [40]:
# Create Country Chart
@pn.cache
def create_bar_chart_country(df_eu_total, df_country, country):
    # Calculate the average renewable energy percentage for all years (used for annotation)
    eu_total_avg = df_eu_total['Renewable Percentage'].mean()
    
    # Map EU total average to a color in the Viridis scale
    color_scale = px.colors.sequential.Viridis
    normalized_avg = eu_total_avg / 100  # Normalize average to a 0-1 range
    scaled_color = sample_colorscale(color_scale, normalized_avg)[0]

    # Create the figure with Plotly
    fig = go.Figure()

    # Add the line trace for EU total renewable energy percentage over the years
    fig.add_trace(go.Scatter(
        x=df_eu_total['Year'], 
        y=df_eu_total['Renewable Percentage'], 
        mode='lines+markers',
        line=dict(color=scaled_color, width=3),
        marker=dict(
            size=6,
            color=df_eu_total['Renewable Percentage'],  # Map the renewable percentage to color
            colorscale='Viridis',
            coloraxis="coloraxis",
        ),
        hovertemplate="🇪🇺 %{x}: <b>%{y:.1f}%</b>",
        name="EU Total",
    ))

    # Add the bar trace for the selected country's renewable energy percentage
    fig.add_trace(go.Bar(
        x=df_country['Year'], 
        y=df_country['Renewable Percentage'],
        marker=dict(
            color=df_country['Renewable Percentage'],  # Map the renewable percentage to color
            colorscale='Viridis',
            coloraxis="coloraxis",
        ),
        customdata=df_country[['Country Flag', 'Country']].values,  # Attach custom data
        hovertemplate="Country: %{customdata[0]} "
                      "<b>%{customdata[1]}</b><br>"
                      "Year: <b>%{x}</b><br>"
                      "Renewable Percentage: <b>%{y:.1f}%</b>",
        name="",
        showlegend=False,
    ))

    # Define the layout with legend at the bottom
    fig.update_layout(
        title=f"Renewable Energy Percentage ({country}, 2004-2022)",
        xaxis=dict(
            title="Year",
            showgrid=False,
            showline=True,
            tickmode="linear",
            tick0=2005,
            dtick=5,
            range=[2003.5, 2022.5],
        ),
        yaxis=dict(
            title="Renewable Energy (%)",
        ),
        coloraxis=dict(
            colorscale='Viridis',
            cmin=0,
            cmax=100,
            colorbar=dict(
                orientation="v",
                tickvals=[0, 20, 40, 60, 80, 100],
                ticktext=["0%", "20%", "40%", "60%", "80%", "100%"],
            ),
        ),
        height=450,
        margin={"t": 50, "b": 50, "l": 50, "r": 50},
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1-0.01,
            xanchor="left",
            x=0+0.01
        ),
    )

    return fig

In [41]:
# Create chart
country = selected_country
fig_by_country = create_bar_chart_country(df_eu_total, df_country, country)

In [ ]:
# Display as Panel pane
fig_pane_2 = pn.pane.Plotly(fig_by_country, width=1200)
fig_pane_2

BokehModel(combine_events=True, render_bundle={'docs_json': {'65d07214-8ad0-47b6-81af-146b6417d6f3': {'version…

#### Comparison Charts

In [43]:
# @pn.cache
# def create_line_chart_eu_total(df_eu_total):
#     # Calculate the average renewable energy percentage for all years (used for annotation)
#     eu_total_avg = df_eu_total['Renewable Percentage'].mean()
    
#     # Map EU total average to a color in the Viridis scale
#     color_scale = px.colors.sequential.Viridis
#     normalized_avg = eu_total_avg / 100  # Normalize average to a 0-1 range
#     scaled_color = px.colors.sample_colorscale(color_scale, normalized_avg)[0]

#     # Create the line chart with Plotly go
#     fig = go.Figure()

#     # Add the line trace for Renewable Energy Percentage over the years
#     fig.add_trace(go.Scatter(
#         x=df_eu_total['Year'],  # x values: Year
#         y=df_eu_total['Renewable Percentage'],  # y values: Renewable Percentage
#         mode='lines+markers',  # Show both the line and markers
#         line=dict(color=scaled_color, width=3),
#         marker=dict(
#             size=6,
#             color=df_eu_total['Renewable Percentage'],  # Map the renewable percentage to color
#             colorscale='Viridis',
#             coloraxis="coloraxis",
#         ),
#         hovertemplate="🇪🇺 %{x}: <b>%{y:.1f}% </b>",
#         name="",
#     ))

#     # Define the layout
#     fig.update_layout(
#         title="EU Total Renewable Energy Percentage Over Time (2004-2022)",
#         xaxis=dict(
#             title="Year",
#             showgrid=False,
#             showline=True,
#             ticks="outside",
#             tickmode="linear",
#             tick0=2005,
#             dtick=5,
#             range=[2005, 2022],
#             domain=[0, 1],
#         ),
#         yaxis=dict(
#             title="Renewable Energy (%)",
#             ticks="outside",
#             tickmode="linear",
#             tick0=0,
#             dtick=5,
#         ),
#         coloraxis=dict(
#             colorscale='Viridis',
#             cmin=0,
#             cmax=100,
#             colorbar=dict(
#                 orientation="v",
#                 tickvals=[0, 20, 40, 60, 80, 100],
#                 ticktext=["0%", "20%", "40%", "60%", "80%", "100%"],
#             ),
#         ),
#         height=450,
#         margin={"t": 50, "b": 50, "l": 10, "r": 10},
#         showlegend=False,
#     )

#     return fig


In [44]:
# # Create the line chart for EU total
# fig_line = create_line_chart_eu_total(df_eu_total)

In [45]:
# # Display as Panel pane
# fig_pane_3 = pn.pane.Plotly(fig_line, sizing_mode='stretch_width')
# fig_pane_3

## Create Dashboard

#### Update Map & Charts

In [46]:
# Map and bar chart update functions
@pn.cache
def update_map(year):
    df_selected_year = df_renewable[df_renewable['Year'] == year]
    return create_choropleth_map(df_selected_year)

@pn.cache
def update_year_chart(year):
    df_selected_year = df_renewable[df_renewable['Year'] == year].sort_values(by='Renewable Percentage')
    return create_bar_chart_year(df_selected_year, year)

@pn.cache
def update_country_chart(country):
    df_selected_country = df_renewable[df_renewable['Country'] == country]
    return create_bar_chart_country(df_eu_total, df_selected_country, country)

def update_country_chart(selected_value):
    # Extract the country name from the selection
    selected_country = selected_value.split(" ", 1)[1]
    
    # Filter the dataframe for the selected country
    df_selected_country = df_renewable[df_renewable['Country'] == selected_country]
    
    # Create the bar chart
    return create_bar_chart_country(df_eu_total, df_selected_country, selected_country)


#### Bind Functions

In [47]:
# Bind year selection
interactive_map = pn.bind(update_map, year_slider)
interactive_bar_chart_year = pn.bind(update_year_chart, year_slider)

In [48]:
# Bind country selection
interactive_bar_chart_country = pn.bind(update_country_chart, country_selection.param.value)

#### Panes & Tabs

In [53]:
filter_year_pane = pn.Column(
    year_slider, 
    pn.pane.Plotly(interactive_bar_chart_year, width=1200)
)

filter_country_pane = pn.Column(
    country_selection, 
    pn.pane.Plotly(interactive_bar_chart_country, width=1200)
)

In [54]:
tabs = pn.Tabs(
    ('Filter by Year', filter_year_pane),
    ('Filter by Country', filter_country_pane)
)

tabs

BokehModel(combine_events=True, render_bundle={'docs_json': {'efafd5ed-2f7a-48a3-a39e-cdbc4ac76b20': {'version…

#### Layout Components

In [56]:
layout = pn.Column(
    pn.pane.Plotly(interactive_map, width=1200),
    tabs,
)

layout

BokehModel(combine_events=True, render_bundle={'docs_json': {'8a9c6ae0-8dd8-49fa-8891-07636ad52699': {'version…

#### Serve Dashboard

In [57]:
pn.template.FastListTemplate(
    title="EU Energy Map", sidebar=[sidebar_pane],
    main=[layout]
).servable();